In [0]:
!pip install -U sentence-transformers

In [0]:
### read dataframe after preprocessing
import pandas as pd
df = pd.read_csv('/content/drive/My Drive/Deep Learning Data/Covid-19 Text Mining/clean_df.csv', index_col = 0)
df.head()

In [0]:
### We are gonna use the abstract as summarization of papers, thus we need to further remove the NaN
df = df.dropna()

In [0]:
df.head()

In [0]:
import re

df['body_text'] = df['body_text'].apply(lambda x: re.sub('[^a-zA-z0-9\s]','',x))
df['abstract'] = df['abstract'].apply(lambda x: re.sub('[^a-zA-z0-9\s]','',x))

In [0]:
def lower_case(input_str):
    input_str = input_str.lower()
    return input_str

df['body_text'] = df['body_text'].apply(lambda x: lower_case(x))
df['abstract'] = df['abstract'].apply(lambda x: lower_case(x))

In [0]:
df.head(4)

In [0]:
df.to_csv("/content/drive/My Drive/Deep Learning Data/Covid-19 Text Mining/semantic_covid.csv")

In [0]:
df_covid_test = pd.read_csv("/content/drive/My Drive/Deep Learning Data/Covid-19 Text Mining/semantic_covid.csv")
text = df_covid_test.drop(["authors", "journal", "Unnamed: 0"], axis=1)
text.head(5)

In [0]:
text_dict = text.to_dict()
len_text = len(text_dict["paper_id"])

In [0]:
from tqdm import tqdm
paper_id_list  = []
body_text_list = []

title_list = []
abstract_list = []
abstract_summary_list = []
for i in tqdm(range(0,len_text)):
  paper_id = text_dict["paper_id"][i]
  body_text = text_dict["body_text"][i].split("\n")
  title = text_dict["title"][i]
  abstract = text_dict["abstract"][i]
  abstract_summary = text_dict["abstract_summary"][i]
  for b in body_text:
    paper_id_list.append(paper_id)
    body_text_list.append(b)
    title_list.append(title)
    abstract_list.append(abstract)
    abstract_summary_list.append(abstract_summary)

In [0]:
df_sentences = pd.DataFrame({"paper_id":paper_id_list},index=body_text_list)
df_sentences.to_csv("/content/drive/My Drive/Deep Learning Data/Covid-19 Text Mining/semantic_sentences.csv")
df_sentences.head()

In [0]:
df_sentences = pd.DataFrame({"paper_id":paper_id_list,"title":title_list,"abstract":abstract_list,"abstract_summary":abstract_summary_list},index=body_text_list)
df_sentences.to_csv("/content/drive/My Drive/Deep Learning Data/Covid-19 Text Mining/covid_sentences_Full.csv")
df_sentences.head()

In [0]:
df_sentences = df_sentences["paper_id"].to_dict()
df_sentences_list = list(df_sentences.keys())
len(df_sentences_list)

In [0]:
list(df_sentences.keys())[:5]

In [0]:
df_sentences_list = [str(d) for d in tqdm(df_sentences_list)]

In [0]:
import pandas as pd
df = pd.read_csv("/content/drive/My Drive/Deep Learning Data/Covid-19 Text Mining/covid_sentences_Full.csv", index_col=0)
df.head()

In [0]:
#https://github.com/UKPLab/sentence-transformers/blob/master/examples/application_semantic_search.py
"""
This is a simple application for sentence embeddings: semantic search
We have a corpus with various sentences. Then, for a given query sentence,
we want to find the most similar sentence in this corpus.
This script outputs for various queries the top 5 most similar sentences in the corpus.
"""

from sentence_transformers import SentenceTransformer
import scipy.spatial
import pickle as pkl
embedder = SentenceTransformer('bert-base-nli-mean-tokens')

# Corpus with example sentences
corpus = df_sentences_list
corpus_embeddings = embedder.encode(corpus,show_progress_bar=True)
with open("/content/drive/My Drive/BertSentenceSimilarity/Pickles/corpus_embeddings.pkl" , "rb") as file_:
  corpus_embeddings = pkl.load(file_)

# Query sentences:
queries = ['What has been published about medical care?',
           'Knowledge of the frequency, manifestations, and course of extrapulmonary manifestations of COVID-19, including, but not limited to, possible cardiomyopathy and cardiac arrest',
           'Use of AI in real-time health care delivery to evaluate interventions, risk factors, and outcomes in a way that could not be done manually',
           'Resources to support skilled nursing facilities and long term care facilities.',
           'Mobilization of surge medical staff to address shortages in overwhelmed communities .',
           'Age-adjusted mortality data for Acute Respiratory Distress Syndrome (ARDS) with/without other organ failure – particularly for viral etiologies .']
query_embeddings = embedder.encode(queries,show_progress_bar=True)

# Find the closest 5 sentences of the corpus for each query sentence based on cosine similarity
closest_n = 5
print("\nTop 5 most similar sentences in corpus:")
for query, query_embedding in zip(queries, query_embeddings):
    distances = scipy.spatial.distance.cdist([query_embedding], corpus_embeddings, "cosine")[0]

    results = zip(range(len(distances)), distances)
    results = sorted(results, key=lambda x: x[1])

    print("\n\n=========================================================")
    print("==========================Query==============================")
    print("===",query,"=====")
    print("=========================================================")


    for idx, distance in results[0:closest_n]:
        print("Score:   ", "(Score: %.4f)" % (1-distance) , "\n" )
        print("Paragraph:   ", corpus[idx].strip(), "\n" )
        row_dict = df.loc[df.index== corpus[idx]].to_dict()
        print("paper_id:  " , row_dict["paper_id"][corpus[idx]] , "\n")
        print("Title:  " , row_dict["title"][corpus[idx]] , "\n")
        print("Abstract:  " , row_dict["abstract"][corpus[idx]] , "\n")
        print("Abstract_Summary:  " , row_dict["abstract_summary"][corpus[idx]] , "\n")
        print("-------------------------------------------")